In [ ]:
import sys
import pandas as pd
import numpy as np

def topsis(input_file, weights, impacts, result_file):
    try:
        mydata = pd.read_csv(input_file)
        validate_numeric_columns(mydata)

        weights, impacts = parse_weights_and_impacts(weights, impacts, mydata)
        validate_weights_and_impacts(weights, impacts, mydata)

        data = mydata.iloc[:, 1:]
        normalized_data = normalize_data(data)
        weighted_normalized_data = weight_normalized_data(normalized_data, weights)

        ideal_best, ideal_worst = compute_ideal_values(weighted_normalized_data, impacts)
        separation_best = compute_separation(normalized_data, ideal_best)
        separation_worst = compute_separation(normalized_data, ideal_worst)

        topsis_score = calculate_topsis_score(separation_worst, separation_best)
        rank = calculate_rank(topsis_score)

        result_data = create_result_dataframe(mydata, topsis_score, rank)
        save_result_to_csv(result_data, result_file)
        print(f"Result saved to {result_file}")

    except FileNotFoundError:
        print("Error: File not found.")

def validate_numeric_columns(mydata):
    for col in mydata.columns:
        try:
            mydata[col] = pd.to_numeric(mydata[col])
        except ValueError:
            print(f"Error: Non-numeric values found in column '{col}'.")
            raise ValueError("Invalid input file format")

def parse_weights_and_impacts(weights, impacts, mydata):
    weights = [float(w) for w in weights.split(',')]
    impacts = [i.strip().lower() for i in impacts.split(',')]
    return weights, impacts

def validate_weights_and_impacts(weights, impacts, mydata):
    if len(weights) != len(impacts) or len(weights) != len(mydata.columns) - 1:
        print("Error: Number of weights, impacts, and columns must be the same.")
        raise ValueError("Invalid weights or impacts")

    if not all(impact in ['+', '-'] for impact in impacts):
        print("Error: Impacts must be either +ve or -ve.")
        raise ValueError("Invalid impacts")

def normalize_data(data):
    return data.iloc[:, 1:].apply(lambda val: val / np.sqrt(np.sum(val**2)))

def weight_normalized_data(normalized_data, weights):
    weighted_normalized_data = normalized_data.copy()
    for i in range(1, len(weights)):
        weighted_normalized_data[data.columns[i]] *= weights[i-1]
    return weighted_normalized_data

def compute_ideal_values(weighted_normalized_data, impacts):
    ideal_best, ideal_worst = [], []
    for i in range(len(data.columns)):
        if impacts[i] == '+':
            ideal_best = pd.to_numeric(weighted_normalized_data.max())
            ideal_worst = pd.to_numeric(weighted_normalized_data.min())
        else:
            ideal_worst = pd.to_numeric(weighted_normalized_data.max())
            ideal_best = pd.to_numeric(weighted_normalized_data.min())
    return ideal_best, ideal_worst

def compute_separation(normalized_data, ideal_values):
    return np.sqrt(np.sum((normalized_data - ideal_values.values) ** 2, axis=1))

def calculate_topsis_score(separation_worst, separation_best):
    return separation_worst / (separation_worst + separation_best)

def calculate_rank(topsis_score):
    return pd.Series(topsis_score).rank(ascending=False)

def create_result_dataframe(mydata, topsis_score, rank):
    result_data = mydata.copy()
    result_data['Topsis Score'] = topsis_score
    result_data['Rank'] = rank
    return result_data

def save_result_to_csv(result_data, result_file):
    result_data.to_csv(result_file, index=False)

if __name__ == "__main__":
    if len(sys.argv) != 5:
        print("Error: Incorrect number of arguments.")
        print("Usage: python topsis.py <inputFileName> <Weights> <Impacts> <resultFileName>")
    else:
        topsis(sys.argv[1], sys.argv[2], sys.argv[3], sys.argv[4])
